In [81]:
import functions
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from sklearn.feature_selection import f_classif, mutual_info_classif
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [82]:
with open("./Dataset/x_test.txt") as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/x_train.txt") as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/y_train.txt") as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]

In [88]:
print("X_test datapoints:", len(X_test), " features:", len(X_test[0]))
print("X_train datapoints:", len(X_train), " features:", len(X_train[0]))
print("y_train datapoints:", len(y_train))

X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000


In [89]:
# MLPClassifier
solver = ["adam", "sgd"]
activation = ["relu", "logistic", "tanh"]
hidden_layer_sizes = [(100,), (200,), (300,)]
mlpClassifierParameters = functions.generateParameters([solver, activation, hidden_layer_sizes], globals())

# Voting?


models = [
    {"model": functions.ModelType.MLPClassifier, "parameters": mlpClassifierParameters},
]

In [90]:
# KBest
k = [2,3,4,5,8]
score_func = [f_classif, mutual_info_classif]
kBestParameters = functions.generateParameters([score_func, k], globals())


featureSelectors = [
    {"model": functions.FeatureSelectorType.KBest, "parameters": kBestParameters},
]

In [91]:
y_train_ravel = np.ravel(y_train, order="C")
y_train_ravel = y_train_ravel.astype(int)

In [92]:
RESULTS_FILENAME = "./Results/Limited/results-salveen2"

In [93]:
%%time
results = functions.conductExperiments(
    models=models,
    featureSelectors=featureSelectors,
    X_orig=X_train,
    y_orig=y_train_ravel,
    getLimitedScore=True,
)
with open(RESULTS_FILENAME, "wb") as f:
    pickle.dump(results, f)

Performed Experiment 1/180(approx) took (s): model: 0.36 selector 0.14 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 2} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (100,)}
Performed Experiment 2/180(approx) took (s): model: 0.49 selector 0.14 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 2} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (200,)}
Performed Experiment 3/180(approx) took (s): model: 0.56 selector 0.14 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 2} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (300,)}
Performed Experiment 4/180(approx) took (s): model: 0.46 selector 0.14 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 2} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logis

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 73/180(approx) took (s): model: 1.05 selector 0.19 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (100,)}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 74/180(approx) took (s): model: 2.37 selector 0.19 with: FeatureSelectorType.KBest {'score_func': <function f_classif at 0x15cf7f1a0>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (200,)}


...

Performed Experiment 164/180(approx) took (s): model: 2.02 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (200,)}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 165/180(approx) took (s): model: 3.13 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'relu', 'hidden_layer_sizes': (300,)}
Performed Experiment 166/180(approx) took (s): model: 0.32 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logistic', 'hidden_layer_sizes': (100,)}
Performed Experiment 167/180(approx) took (s): model: 0.39 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'logistic', 'hidden_layer_sizes': (200,)}
Performed Experiment 168/180(approx) took (s): model: 0.6 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelTyp

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Performed Experiment 171/180(approx) took (s): model: 4.91 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'adam', 'activation': 'tanh', 'hidden_layer_sizes': (300,)}
Performed Experiment 172/180(approx) took (s): model: 0.7 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (100,)}
Performed Experiment 173/180(approx) took (s): model: 1.61 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClassifier {'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (200,)}
Performed Experiment 174/180(approx) took (s): model: 2.96 selector 3.39 with: FeatureSelectorType.KBest {'score_func': <function mutual_info_classif at 0x15dabe700>, 'k': 8} ModelType.MLPClass

In [94]:
with open(RESULTS_FILENAME, "rb") as input_file:
    results = pickle.load(input_file)

In [95]:
resultsDf = pd.DataFrame(
    results,
    columns=functions.RESULTS_COLUMNS,
)
resultsDf = resultsDf.sort_values(by=["score"], ascending=False)
resultsDf

,score,numberOfTruePositives,accuracy,precision,numberOfFeatures,model,model_parameters,feature_selector,selector_parameters
98,0.555556,380,0.570303,0.582946,2,MLPClassifier,"{'solver': 'adam', 'activation': 'tanh', 'hidd...",KBest,{'score_func': <function mutual_info_classif a...
92,0.552288,378,0.567879,0.570451,2,MLPClassifier,"{'solver': 'adam', 'activation': 'relu', 'hidd...",KBest,{'score_func': <function mutual_info_classif a...
96,0.552288,378,0.567879,0.579353,2,MLPClassifier,"{'solver': 'adam', 'activation': 'tanh', 'hidd...",KBest,{'score_func': <function mutual_info_classif a...
91,0.547386,375,0.563636,0.569971,2,MLPClassifier,"{'solver': 'adam', 'activation': 'relu', 'hidd...",KBest,{'score_func': <function mutual_info_classif a...
101,0.545752,374,0.553939,0.549875,2,MLPClassifier,"{'solver': 'sgd', 'activation': 'relu', 'hidde...",KBest,{'score_func': <function mutual_info_classif a...
...,...,...,...,...,...,...,...,...,...
166,0.248366,312,0.488485,0.488294,8,MLPClassifier,"{'solver': 'adam', 'activation': 'logistic', '...",KBest,{'score_func': <function mutual_info_classif a...
174,0.248366,312,0.499394,0.494703,8,MLPClassifier,"{'solver': 'sgd', 'activation': 'logistic', 'h...",KBest,{'score_func': <function mutual_info_classif a...
165,0.246732,311,0.471515,0.468182,8,MLPClassifier,"{'solver': 'adam', 'activation': 'logistic', '...",KBest,{'score_func': <function mutual_info_classif a...
178,0.238562,306,0.472121,0.468643,8,MLPClassifier,"{'solver': 'sgd', 'activation': 'tanh', 'hidde...",KBest,{'score_func': <function mutual_info_classif a...


,score,numberOfTruePositives,accuracy,precision,numberOfFeatures,model,model_parameters,feature_selector,selector_parameters
98,0.555556,380,0.570303,0.582946,2,MLPClassifier,"{'solver': 'adam', 'activation': 'tanh', 'hidd...",KBest,{'score_func': <function mutual_info_classif a...
